implement the Item-Based Collaborative Memory Network (ICMN) recommendation system:

In [ ]:
import pandas as pd
df = pd.read_csv("/content/ratings.dat",sep="::", header=None, names=["UserID", "MovieID", "Rating", "TimeStamp"])

<ipython-input-11-230f9cb2a084>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("/content/ratings.dat",sep="::", header=None, names=["UserID", "MovieID", "Rating", "TimeStamp"])


In [ ]:
df.head()

,UserID,MovieID,Rating,TimeStamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
import numpy as np

class ICMN:
    def __init__(self, num_items, embedding_dim, alpha, beta):
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.alpha = alpha
        self.beta = beta
        self.item_embeddings = np.random.randn(num_items, embedding_dim)
        self.item_similarities = np.zeros((num_items, num_items))

    def compute_item_similarities(self, interactions):
      # Compute co-occurrence matrix
      cooccurrence_matrix = np.zeros((self.num_items, self.num_items))
      for user_interactions in interactions.values():
          for i in range(len(user_interactions)):
              for j in range(i + 1, len(user_interactions)):
                  item_i = user_interactions[i]
                  item_j = user_interactions[j]
                  cooccurrence_matrix[item_i, item_j] += 1
                  cooccurrence_matrix[item_j, item_i] += 1
      # Compute item similarities using cosine similarity
      norms = np.linalg.norm(cooccurrence_matrix, axis=1)

      for i in range(self.num_items):
          for j in range(i + 1, self.num_items):
              similarity = np.dot(cooccurrence_matrix[i], cooccurrence_matrix[j]) / (norms[i] * norms[j])
              self.item_similarities[i, j] = similarity
              self.item_similarities[j, i] = similarity


    def update_item_embeddings(self, interactions):
        for user_interactions in interactions.values():
            for i in range(len(user_interactions) - 1):
                item_i = user_interactions[i]
                item_j = user_interactions[i + 1]
                error = self.item_embeddings[item_i] - self.alpha * self.item_embeddings[item_j]
                self.item_embeddings[item_i] -= self.beta * error

    def fit(self, interactions, num_epochs):
        for epoch in range(num_epochs):
            print(f"Epoch {epoch + 1}/{num_epochs}")
            self.compute_item_similarities(interactions)
            self.update_item_embeddings(interactions)

    def recommend_items(self, user_interactions, top_k):
        user_embeddings = np.mean(self.item_embeddings[user_interactions], axis=0)
        item_scores = np.dot(user_embeddings, self.item_embeddings.T)
        top_items = np.argsort(item_scores)[::-1][:top_k]
        return top_items

    def compute_loss(self, interactions):
      total_loss = 0
      num_predictions = 0

      for user_id, user_interactions in interactions.items():
          for i in range(len(user_interactions) - 1):
              item_i = user_interactions[i]
              item_j = user_interactions[i + 1]
              predicted_error = self.item_embeddings[item_i] - self.alpha * self.item_embeddings[item_j]
              actual_error = self.item_embeddings[item_i] - self.item_embeddings[item_j]
              rating = interactions[user_id][i + 1]  # Fetch the rating from the interactions dictionary
              squared_error = np.sum((predicted_error * rating - actual_error) ** 2)
              total_loss += squared_error
              num_predictions += 1

      mse_loss = total_loss / num_predictions
      rmse_loss = np.sqrt(mse_loss)
      return rmse_loss



def load_ratings(filename):
    interactions = {}

    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split("::")
            if len(parts) != 4:
                continue

            user_id, item_id, rating, timestamp = parts
            user_id = int(user_id)
            item_id = int(item_id)

            if user_id not in interactions:
                interactions[user_id] = []

            interactions[user_id].append(item_id)

    return interactions


def load_movies(filename):
    movies = {}

    with open(filename, 'r', encoding='latin1') as file:
        for line in file:
            parts = line.strip().split("::")
            if len(parts) != 3:
                continue

            item_id, title, _ = parts
            item_id = int(item_id)

            movies[item_id] = title

    return movies


filename = '/content/ratings.dat'
interactions = load_ratings(filename)


movies_filename = '/content/movies.dat'
movies = load_movies(movies_filename)

num_items = max(max(items) for items in interactions.values()) + 1
embedding_dim = 20
alpha = 0.5
beta = 0.01

icmn = ICMN(num_items, embedding_dim, alpha, beta)
icmn.fit(interactions, num_epochs=20)

loss = icmn.compute_loss(interactions)
print("Root mean squared error:", loss)


Epoch 1/20


<ipython-input-16-0ab49f621cda>:28: RuntimeWarning: invalid value encountered in double_scalars
  similarity = np.dot(cooccurrence_matrix[i], cooccurrence_matrix[j]) / (norms[i] * norms[j])


Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Root mean squared error: 204.8413555483708


In [ ]:
# Get movie name
user_id = 1
user_interactions = interactions[user_id]
top_k = 5  # Number of rec movies
recommended_items = icmn.recommend_items(user_interactions, top_k)

print("Recommended Movies:")
for item_id in recommended_items:
    movie_name = movies.get(item_id, "Unknown")
    print(f"- {movie_name}")

Recommended Movies:
- Associate, The (L'Associe)(1982)
- Dangerous Game (1993)
- Unknown
- Other Voices, Other Rooms (1997)
- Hungarian Fairy Tale, A (1987)


In [ ]:
user_id = 1
user_interactions = interactions[user_id]
top_k = 3
recommended_items = icmn.recommend_items(user_interactions, top_k)
print("Recommended items:", recommended_items)

Recommended items: [1001  439 1540]


In [ ]:
top_k = 3
# For all users
for user_id, user_interactions in interactions.items():
    recommended_items = icmn.recommend_items(user_interactions, top_k)
    recommended_movie_names = [movies.get(item_id, "Unknown") for item_id in recommended_items]
    print(f"Recommended items for user {user_id}: {recommended_movie_names}")

Streaming output truncated to the last 5000 lines.
Recommended items for user 1041: ['Tickle in the Heart, A (1996)', 'Unknown', 'Anguish (Angustia) (1986)']
Recommended items for user 1042: ['Unknown', 'Caught Up (1998)', 'Night Tide (1961)']
Recommended items for user 1043: ['Nô (1998)', 'Salut cousin! (1996)', 'Tess of the Storm Country (1922)']
Recommended items for user 1044: ['Legend of Lobo, The (1962)', 'Silence of the Palace, The (Saimt el Qusur) (1994)', 'Buddy Boy (1999)']
Recommended items for user 1045: ['Desert Winds (1995)', 'Nô (1998)', 'Let it Come Down: The Life of Paul Bowles (1998)']
Recommended items for user 1046: ['Quartier Mozart (1992)', 'Nô (1998)', 'Open Season (1996)']
Recommended items for user 1047: ['Boys, Les (1997)', 'Choices (1981)', 'Frank and Ollie (1995)']
Recommended items for user 1048: ['Five Wives, Three Secretaries and Me (1998)', 'Gendernauts (1999)', 'Tess of the Storm Country (1922)']
Recommended items for user 1049: ['Naturally Native (1998